<a href="https://colab.research.google.com/github/EvenSol/testneqsim/blob/master/google_sheet_interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title Use of NeqSim in Googel Sheets
#@
!pip install py4j
!pip install --upgrade -q gspread
!rm -rf testneqsim
!git clone https://github.com/EvenSol/testneqsim.git
!cd testneqsim
!pip install testneqsim/neqsim-0.0.1-py3-none-any.whl
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

Cloning into 'testneqsim'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 52 (delta 4), reused 0 (delta 0), pack-reused 43
Unpacking objects: 100% (52/52), done.


# How to connect and create a Google sheet

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
sheetTitle = 'a new neqsim spread sheet'  #@param {type:"string"}
gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.create(sheetTitle)

# Writing values to a google sheet

In [44]:
# Open our new sheet and add some data.
worksheet = gc.open(sheetTitle).sheet1
worksheet.clear()

componentName = 'methane' #@param {type:"string"}
minPressure = 10.0 #@param {type:"number"}
maxPressure = 30.0 #@param {type:"number"}

fluid1 = fluid('srk') #create a fluid using the SRK-EoS
fluid1.addComponent('propane', 1.0) #adding 1 mole methane to the fluid
fluid1.setTemperature(111.0) # setting an initial temperature of -160 deg C
fluid1.setPressure(1.0)
bubt(fluid1)
print('methane buble point temperature ', fluid1.getTemperature('C'), " at pressure ", fluid1.getPressure('bara'))
print('methane entalpy of vaporization ', (fluid1.getPhase('gas').getEnthalpy('kJ/kg')-fluid1.getPhase('oil').getEnthalpy('kJ/kg'), ' kJ/kg'))

def bubleP(pressure):
    fluid1.setPressure(pressure)
    bubt(fluid1)
    return fluid1.getTemperature('C')

pressure = np.arange(minPressure, maxPressure, 1.0)
temperature = [bubleP(P) for P in pressure]

print(pressure.size)
cell_list = worksheet.range('A2:A'+str(pressure.size+1))
cell_list2 = worksheet.range('B2:B'+str(pressure.size+1))

i=0
for cell in cell_list:
  cell.value = pressure[i]
  i +=1

i=0
for cell in cell_list2:
  cell.value = temperature[i]
  i +=1
  
  
worksheet.update_cells(cell_list)
worksheet.update_cells(cell_list2)


methane buble point temperature  -42.17772086617717  at pressure  1.0
methane entalpy of vaporization  (303.40514081044967, ' kJ/kg')
29


{'spreadsheetId': '1ilOefyPXayYqu6e98sNDkIEPVbNekgUpZgrvaX5-JEI',
 'updatedCells': 29,
 'updatedColumns': 1,
 'updatedRange': "'Ark 1'!B2:B30",
 'updatedRows': 29}